In [1]:
from pyspark import SparkContext
sc=SparkContext(master='local[2]',
               appName='my-spark')
from pyspark.sql import SparkSession
spark=SparkSession(sparkContext=sc)

In [9]:
from pyspark.ml.feature import CountVectorizer,CountVectorizerModel
from pyspark.ml.linalg import Vector

#  Define source DataFrame
df = spark.createDataFrame([
  [0, ["ant", "bat", "cat", "dog", "eel","cat"]],
  [1, ["dog","bat", "ant", "bat","bat"]]
 ]).toDF("id", "words")
df.show(5)

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|[ant, bat, cat, d...|
|  1|[dog, bat, ant, b...|
+---+--------------------+



In [12]:
# Count vectorizer
cvModel=CountVectorizer(inputCol="words",outputCol="features",minDF=1,vocabSize=10).fit(df)
cvModel.vocabulary

['bat', 'cat', 'ant', 'dog', 'eel']

In [13]:
cvDF=cvModel.transform(df)
cvDF.show(2,False)

+---+------------------------------+-------------------------------------+
|id |words                         |features                             |
+---+------------------------------+-------------------------------------+
|0  |[ant, bat, cat, dog, eel, cat]|(5,[0,1,2,3,4],[1.0,2.0,1.0,1.0,1.0])|
|1  |[dog, bat, ant, bat, bat]     |(5,[0,2,3],[3.0,1.0,1.0])            |
+---+------------------------------+-------------------------------------+



In [22]:
# Tf IDF
from pyspark.ml.feature import HashingTF
hashtf=HashingTF(numFeatures=16,inputCol="words",outputCol="features(numFeatures),[index],[term frequency]").\
transform(cvDF).show(truncate=False)


+---+------------------------------+-------------------------------------+----------------------------------------------+
|id |words                         |features                             |features(numFeatures),[index],[term frequency]|
+---+------------------------------+-------------------------------------+----------------------------------------------+
|0  |[ant, bat, cat, dog, eel, cat]|(5,[0,1,2,3,4],[1.0,2.0,1.0,1.0,1.0])|(16,[0,12,14,15],[2.0,1.0,2.0,1.0])           |
|1  |[dog, bat, ant, bat, bat]     |(5,[0,2,3],[3.0,1.0,1.0])            |(16,[12,14,15],[3.0,1.0,1.0])                 |
+---+------------------------------+-------------------------------------+----------------------------------------------+

